# 🏎️ CarRacing AI 학습 노트북

이 노트북에서 PPO 알고리즘으로 자율주행 AI를 학습시킵니다.

**예상 소요 시간**: 30분~1시간 (GPU 사용 시)

---

## 1. GPU 설정 확인

**중요**: 런타임 > 런타임 유형 변경 > 하드웨어 가속기를 **GPU**로 설정하세요!

In [ ]:
# GPU 확인
!nvidia-smi

## 2. Google Drive 마운트 (중요!)

**모델을 영구 보존하기 위해 Google Drive에 저장합니다.**

런타임이 종료되어도 모델이 사라지지 않습니다!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

# Google Drive에 저장할 기본 경로
DRIVE_BASE_PATH = '/content/drive/MyDrive/CarRacing_AI'
os.makedirs(DRIVE_BASE_PATH, exist_ok=True)

print("✅ Google Drive 마운트 완료!")
print(f"📁 저장 경로: {DRIVE_BASE_PATH}")
print("\n💡 이제 모델이 Google Drive에 영구 보존됩니다!")

## 3. 필요한 패키지 설치

CarRacing 환경과 강화학습 라이브러리를 설치합니다.

In [ ]:
# SWIG 먼저 설치 (Box2D 빌드에 필요)
!apt-get update -qq
!apt-get install -y swig build-essential python3-dev

# Box2D 먼저 설치
!pip install box2d-py

# 나머지 패키지 설치
!pip install gymnasium stable-baselines3[extra] shimmy moviepy pygame -q

## 4. 라이브러리 임포트

In [ ]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor  # 에피소드 통계 기록용

print("✅ 라이브러리 임포트 완료!")

## 5. 학습 설정 (Google Drive 경로)

In [ ]:
# 학습 설정
TOTAL_TIMESTEPS = 100000  # 총 학습 스텝 (더 늘리면 성능 향상)
CHECKPOINT_FREQ = 20000   # 체크포인트 저장 빈도
MODEL_NAME = "car_racing_driver"

# PPO 하이퍼파라미터
LEARNING_RATE = 0.0003
N_STEPS = 2048
BATCH_SIZE = 64
N_EPOCHS = 10

# 디렉토리 설정 (Google Drive 경로!)
CHECKPOINT_DIR = f"{DRIVE_BASE_PATH}/checkpoints"
LOG_DIR = f"{DRIVE_BASE_PATH}/logs"
MODEL_DIR = f"{DRIVE_BASE_PATH}/models"

# 디렉토리 생성
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

print("✅ 설정 완료!")
print(f"📁 체크포인트: {CHECKPOINT_DIR}")
print(f"📁 로그: {LOG_DIR}")
print(f"📁 모델: {MODEL_DIR}")
print("\n💾 모든 파일이 Google Drive에 저장됩니다!")

## 6. CarRacing 환경 생성 (Monitor 래퍼 추가)

**Monitor 래퍼를 추가하여 에피소드 통계(ep_rew_mean, ep_len_mean)를 확실하게 기록합니다.**

In [ ]:
print("🏎️ CarRacing 환경 초기화 중...")

# CarRacing-v3 환경 생성 함수 (Monitor 래퍼 포함)
def make_env():
    env = gym.make("CarRacing-v3", continuous=True)
    # Monitor 래퍼: 에피소드 통계를 자동으로 기록
    env = Monitor(env)
    return env

# 벡터 환경 생성
env = DummyVecEnv([make_env])

print("✅ 환경 초기화 완료!")
print("💡 Monitor 래퍼가 추가되어 ep_rew_mean이 출력됩니다!")

## 7. PPO 모델 생성

In [ ]:
print("🤖 PPO 모델 생성 중...")

model = PPO(
    "CnnPolicy",                    # CNN 정책 (이미지 입력용)
    env,
    verbose=1,                      # 학습 진행 상황 출력
    tensorboard_log=LOG_DIR,        # TensorBoard 로그 저장
    learning_rate=LEARNING_RATE,
    n_steps=N_STEPS,
    batch_size=BATCH_SIZE,
    n_epochs=N_EPOCHS
)

print("✅ 모델 생성 완료!")

## 8. 체크포인트 콜백 설정

In [ ]:
checkpoint_callback = CheckpointCallback(
    save_freq=CHECKPOINT_FREQ,
    save_path=CHECKPOINT_DIR,
    name_prefix=MODEL_NAME
)

print("✅ 체크포인트 콜백 설정 완료!")

## 9. 학습 시작

이제 학습을 시작합니다! 진행 상황을 실시간으로 볼 수 있습니다.

**참고**: 
- 초반에는 점수가 음수입니다 (차가 제대로 못 움직임)
- 학습이 진행되면서 점점 점수가 올라갑니다
- 점수가 0을 넘어 양수가 되면 운전을 시작한 것입니다!

**중요**: 학습 중에는 절대 런타임을 종료하지 마세요!

In [ ]:
print("="*50)
print("🏁 훈련 시작!")
print("="*50)
print(f"총 학습 스텝: {TOTAL_TIMESTEPS:,}")
print(f"체크포인트 저장 빈도: 매 {CHECKPOINT_FREQ:,} 스텝")
print(f"📁 저장 위치: Google Drive (MyDrive/CarRacing_AI/)")
print(f"\n목표: 점수가 0점을 넘어 양수가 되면 운전을 시작한 것입니다!")
print("="*50 + "\n")

try:
    model.learn(
        total_timesteps=TOTAL_TIMESTEPS,
        callback=checkpoint_callback
    )
    
    print("\n" + "="*50)
    print("🎉 훈련 완료!")
    print("="*50)
    
except KeyboardInterrupt:
    print("\n\n⚠️ 훈련이 사용자에 의해 중단되었습니다.")

## 10. 최종 모델 저장 및 확인

In [ ]:
final_model_path = os.path.join(MODEL_DIR, f"final_{MODEL_NAME}")
model.save(final_model_path)
print(f"\n💾 최종 모델 저장 완료: {final_model_path}")

# 환경 종료
env.close()

# 저장된 파일 확인
import glob
checkpoints = glob.glob(f"{CHECKPOINT_DIR}/*.zip")
print(f"\n✅ 저장된 체크포인트: {len(checkpoints)}개")
print(f"✅ 최종 모델: {final_model_path}.zip")
print(f"\n📁 Google Drive에서 확인: MyDrive/CarRacing_AI/")
print("\n🎉 모든 작업 완료! 모델이 영구 보존되었습니다!")

## 11. 모델 다운로드 (선택사항)

학습된 모델을 로컬 PC로 다운로드하려면 아래 셀을 실행하세요.

In [ ]:
from google.colab import files

# 최종 모델 다운로드
files.download(f"{final_model_path}.zip")
print("📥 모델 다운로드 시작!")

## 12. TensorBoard로 학습 과정 확인 (선택사항)

학습 완료 후 학습 진행 상황을 그래프로 확인할 수 있습니다.

**⭐ 권장: 방법 1 사용 (간단하고 무료)**

**방법 1: Colab 내장 TensorBoard**
- 아래 "방법 1" 셀 실행
- 10~20초 기다리면 인터페이스 표시
- 무료, 설정 필요 없음

**방법 2: ngrok 터널 (고급 사용자용)**
- ngrok 계정 필요 (무료)
- 학습 중에도 실시간 확인 가능
- **Colab Secrets로 토큰 저장** - 한 번만 설정하면 계속 사용 가능!

---

## 다음 단계

학습이 완료되었습니다! 이제 `colab_test.ipynb`를 실행하여 학습된 AI의 주행을 테스트해보세요.

### 💡 TensorBoard 사용 팁:
- **방법 1**이 안 되면 **방법 2**를 사용하세요
- 학습 **시작 전**에 방법 2를 실행하면 학습 중에도 실시간 확인 가능
- TensorBoard에서 볼 수 있는 것:
  - `rollout/ep_rew_mean`: 평균 보상 (높을수록 좋음)
  - `rollout/ep_len_mean`: 평균 에피소드 길이
  - `train/explained_variance`: 예측 정확도 (0.8 이상이면 우수)
  - `train/value_loss`: 가치 함수 오차 (낮을수록 좋음)

### 학습을 더 개선하려면:
- `TOTAL_TIMESTEPS`를 늘려보세요 (예: 300000)
- 하이퍼파라미터를 조정해보세요
- 여러 환경을 병렬로 실행해보세요 (SubprocVecEnv)

### 문제가 발생한다면:
- GPU가 활성화되어 있는지 확인하세요
- 런타임을 재시작하고 다시 실행해보세요
- Google Drive 용량을 확인하세요 (로그 파일이 용량을 많이 차지함)
- 체크포인트에서 학습을 재개할 수 있습니다

In [ ]:
# ===== 방법 1: Colab 내장 TensorBoard (권장) =====
%load_ext tensorboard
%tensorboard --logdir {LOG_DIR}

In [ ]:
# ===== 방법 2: ngrok 터널 - Colab Secrets 사용 (권장) =====

# 🔐 Colab Secrets 설정 방법:
# 1. 왼쪽 사이드바에서 🔑 (Secrets) 아이콘 클릭
# 2. "Add new secret" 클릭
# 3. Name: NGROK_TOKEN
# 4. Value: 본인의 ngrok authtoken 입력 (https://dashboard.ngrok.com/get-started/your-authtoken)
# 5. 이 노트북에 대한 액세스 허용 토글 켜기

import subprocess
import time

try:
    # TensorBoard를 백그라운드로 실행
    !pkill -f tensorboard
    subprocess.Popen(['tensorboard', '--logdir', LOG_DIR, '--host', '0.0.0.0', '--port', '6006'])
    time.sleep(5)
    
    # ngrok 설치 및 설정
    !pip install pyngrok -q
    from pyngrok import ngrok
    from google.colab import userdata
    
    # Colab Secrets에서 토큰 가져오기
    try:
        ngrok_token = userdata.get('NGROK_TOKEN')
        ngrok.set_auth_token(ngrok_token)
    except userdata.SecretNotFoundError:
        print("❌ Colab Secrets에 NGROK_TOKEN이 설정되지 않았습니다!")
        print("\n📝 설정 방법:")
        print("1. 왼쪽 사이드바에서 🔑 (Secrets) 클릭")
        print("2. 'Add new secret' 클릭")
        print("3. Name: NGROK_TOKEN")
        print("4. Value: 본인의 ngrok authtoken")
        print("   (https://dashboard.ngrok.com/get-started/your-authtoken)")
        print("5. 이 노트북에 대한 액세스 허용")
        raise
    
    # 기존 터널 종료
    ngrok.kill()
    
    # 새 터널 생성
    public_url = ngrok.connect(6006)
    
    print("="*60)
    print("🎉 TensorBoard가 실행되었습니다!")
    print("="*60)
    print(f"\n🔗 아래 URL을 클릭하세요:")
    print(f"{public_url}")
    print(f"\n💡 이 URL은 학습 중에도 계속 사용 가능합니다!")
    print("="*60)
    
except Exception as e:
    print("❌ ngrok 실행 실패!")
    print(f"에러: {str(e)[:200]}")
    print("\n또는 간단하게 '방법 1'을 사용하세요! (권장)")
    print("="*60)